# Data Cleaning for SET Data(Continue)

Date: 07/05/2019

## Unit and Data Import

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [4]:
## Import data set
cleaned_filepath = 'SET data archive 3416 cases recd thru end of 03 2019.csv'

df = pd.read_csv(cleaned_filepath)
df.head()

,MERGE_DATE,archive_upload_2229,FULL_GRYD_ID_FB,Processing_status,Manual_notes_ETO,no_intake_question,no_retest1_question,no_retest2_question,FEEDBACK_STATUS,date_feedback_uploaded_ETO,...,othgpnorms_4bins,othgpcohe_8bins,othgpID_8bins,othgpemo_8bins,LN_CRIME,LN_VIOLENCE,SETDirections_ETO,SiteName_ETO,PROGRAM_NAME_ETO,enroll_ETO_num
0,2/8/2019,,77th 2-13-1494-1,2.0,"Missing response(s) S11a text, s11b, x10 and x15",,,,2.0,2/8/2019,...,1,3,5,5,0,0,If individual is still receiving services in 6...,1,2,1
1,1/16/2015,,RPARSA-2-64-1,9.0,,,,,0.0,,...,1,4,5,3,0,1.6094379124341,,,,1
2,4/28/2014,,RPARSA-2-77-1,9.0,,,,,0.0,,...,9,9,9,9,1.79175946922805,.693147180559945,,1,,1
3,3/18/2019,,Northeast-25-68-2,1.0,,,,,2.0,3/18/2019,...,9,9,9,9,1.09861228866811,1.38629436111989,If individual is still receiving services in 6...,1,14,1
4,3/18/2019,,Northeast-25-69-2,1.0,,,,,2.0,3/18/2019,...,9,9,9,9,.693147180559945,.693147180559945,If individual is still receiving services in 6...,1,14,1


In [145]:
df['no_fam_num'].value_counts()

1    2020
1    1350
0      23
0      18
        5
Name: no_fam_num, dtype: int64

In [146]:
df.shape

(3416, 432)

## Investigating Data

### Drop the wrong version 

In [147]:
df = df[df['SET_version_num'] == 3]
df['SET_version_num'].value_counts()

3    2800
Name: SET_version_num, dtype: int64

In [148]:
df.head()

,MERGE_DATE,archive_upload_2229,FULL_GRYD_ID_FB,Processing_status,Manual_notes_ETO,no_intake_question,no_retest1_question,no_retest2_question,FEEDBACK_STATUS,date_feedback_uploaded_ETO,...,othgpnorms_4bins,othgpcohe_8bins,othgpID_8bins,othgpemo_8bins,LN_CRIME,LN_VIOLENCE,SETDirections_ETO,SiteName_ETO,PROGRAM_NAME_ETO,enroll_ETO_num
0,2/8/2019,,77th 2-13-1494-1,2.0,"Missing response(s) S11a text, s11b, x10 and x15",,,,2.0,2/8/2019,...,1,3,5,5,0,0,If individual is still receiving services in 6...,1,2,1
3,3/18/2019,,Northeast-25-68-2,1.0,,,,,2.0,3/18/2019,...,9,9,9,9,1.09861228866811,1.38629436111989,If individual is still receiving services in 6...,1,14,1
4,3/18/2019,,Northeast-25-69-2,1.0,,,,,2.0,3/18/2019,...,9,9,9,9,.693147180559945,.693147180559945,If individual is still receiving services in 6...,1,14,1
5,3/18/2019,,Northeast-25-70-2,1.0,,,,,2.0,3/18/2019,...,9,9,9,9,1.09861228866811,.693147180559945,If individual is still receiving services in 6...,1,14,1
6,4/14/2017,1,Olympic-12-39-2,3.0,ETO continues to show duplicate GRYD IDs - the...,2,,,2.0,6/8/2018,...,1,5,2,3,1.38629436111989,1.6094379124341,If individual is still receiving services in 6...,1,17,2


### Creating Unique ID

In [6]:
#showing how many ETO_IDs are missing

df['ETO_ID_FB'] = pd.to_numeric(df['ETO_ID_FB'], downcast = 'integer', errors = 'coerce')
df['ETO_ID_FB'].isnull().value_counts()

#None of them are missing

False    3410
True        6
Name: ETO_ID_FB, dtype: int64

In [7]:
# Change the column name to UniqueID

In [8]:
df.rename(columns={'ETO_ID_FB':'UniqueID'}, inplace = True)

### Column Processing_status

In [9]:
#Drop incomplete processing status
df = df[df['Processing_status'] < 2]

In [10]:
df.head()
df['Processing_status'].value_counts()

1.50    2200
1.00    1123
1.75      26
Name: Processing_status, dtype: int64

## Questionnaire part

### Family Section (clean data for no family people )

In [154]:
#list the questions about family
f_que = df.loc[:,'FS90_younger':'FS90_older_adults'].columns.tolist()
f_que.extend(df.loc[:,'FS90_younger':'FS90_older_adults'].columns.tolist())

#list the index for those who don't have family
df['no_fam_num'] = pd.to_numeric(df['no_fam_num'], downcast = 'integer', errors = 'coerce')
NoFamppl = df[df['no_fam_num'] == 0].index.tolist()
NoFamppl.extend(df[df['no_fam_num'].isnull()].index.tolist())
print(NoFamppl)

[54, 97, 99, 243, 604, 606, 607, 626, 938, 1499, 1512, 1537, 1782, 1796, 2217, 2769, 2946, 3124, 3230, 3231, 3233, 3234, 3328, 3329, 1183, 1945]


In [155]:
df['no_group_num'].value_counts()

0    1453
1    1153
      163
Name: no_group_num, dtype: int64

In [156]:
for col in f_que:
    df[col] = pd.to_numeric(df[col], downcast = 'integer', errors = 'coerce')

In [157]:
#change their responses from f1 to f3 to 0
for index in NoFamppl:
    for col in f_que:
        df.loc[index, col] = 0

In [158]:
f_que.extend(df.loc[:,'F4':'FS98'].columns.tolist())
f_que.extend(df.loc[:,'FS100':'F17x'].columns.tolist())
f_que.extend(df.loc[:,'F18':'F31b'].columns.tolist())

In [159]:
for col in f_que:
    df[col] = pd.to_numeric(df[col], downcast = 'integer', errors = 'coerce')

In [160]:
#change their resposnes from f4 to f31b to nan
for index in NoFamppl:
    for col in f_que:
        df.loc[index, col] = np.nan

### Group Section (clean data for no group people )

In [161]:
#list the index for those who don't have group

df['no_group_num'] = pd.to_numeric(df['no_group_num'], downcast = 'integer', errors = 'coerce')
NoGrppl = df[df['no_group_num'] == 0].index.tolist()
NoGrppl.extend(df[df['no_group_num'].isnull()].index.tolist())

#list the group questions
g_que = df.loc[:,'G2':'G9'].columns.tolist()
g_que.extend(df.loc[:,'G10x':'G23x'].columns.tolist())
g_que.extend(df.loc[:,'G24':'G37'].columns.tolist())

In [162]:
df['no_group_num'].value_counts()

0.0    1453
1.0    1153
Name: no_group_num, dtype: int64

In [163]:
for col in g_que:
    df[col] = pd.to_numeric(df[col], downcast = 'integer', errors = 'coerce')

In [164]:
#change their resposnes from G2 to G37 to nan
for index in NoGrppl:
    for col in g_que:
        df.loc[index, col] = np.nan

### Other Group (O) Section

In [165]:
#change all the columns to numeric

#for col in o_que:
#    df[col] = pd.to_numeric(df[col], downcast = 'integer', errors = 'coerce')

In [166]:
df['no_othgp_num'] = pd.to_numeric(df['no_othgp_num'], downcast = 'integer', errors = 'coerce')
NoOGrppl = df[df['no_othgp_num'] != 1].index.tolist()


In [167]:
for col in o_que:
    for index in NoOGrppl:
        df.loc[index, col] = np.nan

In [168]:
df['O4'].value_counts()

3.0      342
4.0      215
5.0      198
2.0      144
0.0       80
1.0       78
999.0      7
888.0      1
777.0      1
555.0      1
Name: O4, dtype: int64

### Getting rid of Nan

In [171]:
# S section
que = df.loc[:,'S1':'S3'].columns.tolist()
que.extend(df.loc[:,'S4x':'S6ax'].columns.tolist())
que.extend(df.loc[:,'S9':'S10'].columns.tolist())
que.extend(df.loc[:,'S10x':'S11a'].columns.tolist())
que.extend(df.loc[:,'S11b':'S16'].columns.tolist())

# ME,P,C section
que.extend(df.loc[:,'ME17':'C5b'].columns.tolist())

# FS section
que.extend(df.loc[:,'FS89_foster_now':'FS90_older_adults'].columns.tolist())

# F section
que.extend(f_que)

# G section
que.extend(g_que)

# X section
que.extend(df.loc[:,'X18':'X22'].columns.tolist())
que.extend(df.loc[:,'X10':'X17'].columns.tolist())

# O section
o_que = df.loc[:,'O4':'O9'].columns.tolist()
o_que.extend(df.loc[:,'O10':'O15'].columns.tolist())
o_que.extend(df.loc[:,'O16':'O24'].columns.tolist())
que.extend(o_que)
que.extend(['A1','A2','A3'])

# IC section
que.extend(df.loc[:,'IC1':'IC7'].columns.tolist())

# CIW section
que.extend(df.loc[:,'CIW1':'CIW5'].columns.tolist())

In [65]:
print(que)

['S1', 'S2', 'S3', 'S4x', 'S5x', 'S7x', 'S6ax', 'S9', 'S10', 'S10x', 'S10z', 'S11a', 'S11b', 'S12', 'S13', 'S14', 'S16', 'ME17', 'ME18', 'ME19', 'ME20a', 'ME20b', 'ME20x', 'ME22_ER', 'P22', 'P23', 'P24', 'P25', 'P26', 'C1a', 'C1b', 'C2a', 'C2b', 'C3a', 'C3b', 'C4a', 'C4b', 'C5a', 'C5b', 'FS89_foster_now', 'FS89_foster_past', 'FS89_group_now', 'FS89_group_past', 'FS89_adopted_now', 'FS89_adopted_past', 'FS89_parents_now', 'FS89_parents_past', 'FS89_relatives_now', 'FS89_relatives_past', 'FS89_spouse_now', 'FS89_spouse_past', 'FS89_boygirlfriend_now', 'FS89_boygirlfriend_past', 'FS89_alone_now', 'FS89_alone_past', 'FS90_younger', 'FS90_teenagers', 'FS90_20_29_adults', 'FS90_30_49_adults', 'FS90_older_adults', 'FS90_younger', 'FS90_teenagers', 'FS90_20_29_adults', 'FS90_30_49_adults', 'FS90_older_adults', 'FS90_younger', 'FS90_teenagers', 'FS90_20_29_adults', 'FS90_30_49_adults', 'FS90_older_adults', 'F4', 'F5', 'FS91', 'FS92', 'FS93', 'FS94', 'FS95', 'FS96', 'FS97', 'FS98', 'FS100', 'FS1

In [172]:
for col in que:
    df[col].replace('System',np.NaN)  
    df[col] = pd.to_numeric(df[col], downcast = 'integer', errors = 'coerce')
    df.loc[(df[col] == 999), col] = np.nan
    df.loc[(df[col] == 888), col] = np.nan
    df.loc[(df[col] == 777), col] = np.nan
    df.loc[(df[col] == 555), col] = np.nan

## Create new dataframe

In [11]:
df_SETque = df.loc[:,'UniqueID']
df_SETque = df.loc[:,'no_intake_question':'no_retest2_question']
df_SETque['UniqueID'] = df.loc[:,'UniqueID']
cols = df_SETque.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_SETque = df_SETque[cols]

In [20]:
col_needed = df.loc[:,'SET_program_num':'baseline_type'].columns.tolist()
col_needed.extend('enroll_SET_type_num')
col_needed.extend(df.loc[:,'cycle_1_1':'retest'].columns.tolist())

In [24]:
for col in col_needed:
    df_SETque.assign(col = df[col])

KeyError: 'e'

In [188]:
df_SETque.head(20)

,UniqueID,no_intake_question,no_retest1_question,no_retest2_question
3,49866,,,
4,49868,,,
5,49870,,,
8,47002,,,
13,41570,,,
14,42392,,,
15,43298,,,
16,45090,,,
26,26789,,,
27,35390,,,


In [177]:
df_question =df.loc[:,que]
df_question['UniqueID'] = df['UniqueID']
cols = df_question.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_question = df_question[cols]
df_question.head()

,UniqueID,S1,S2,S3,S4x,S5x,S7x,S6ax,S9,S10,...,IC3,IC4,IC5,IC6,IC7,CIW1,CIW2,CIW3,CIW4,CIW5
3,49866,1.0,1.0,0.0,0.0,0.0,40.0,5.0,3.0,0.0,...,3.0,1.0,3.0,3.0,4.0,1.0,4.0,4.0,1.0,3.0
4,49868,0.0,0.0,0.0,0.0,0.0,40.0,5.0,3.0,0.0,...,4.0,1.0,3.0,3.0,4.0,2.0,3.0,4.0,1.0,3.0
5,49870,0.0,1.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,...,4.0,1.0,3.0,3.0,4.0,2.0,3.0,3.0,1.0,3.0
8,47002,0.0,0.0,0.0,30.0,14.0,0.0,6.0,0.0,0.0,...,3.0,4.0,0.0,2.0,4.0,2.0,3.0,1.0,4.0,0.0
13,41570,1.0,0.0,0.0,40.0,5.0,0.0,3.0,0.0,0.0,...,0.0,4.0,2.0,5.0,3.0,NaN,NaN,NaN,NaN,NaN
